# e) Power and Energy Model

Calculates power and energy over the historic daily flow series.

## Procedure

For each day in the series:
- Apply the selected Environmental Assurance Categories for each channel
- Determine the EWR flows for each channel
- Calculate the canal flow
- Calculate the headpond and tailrace levels
- Calculate headlosses in the left channel and canal
- Aportion canal flows amongst the turbines
- Calculate headlosses in the turbines and net head
- Calculate turbine and generator efficiency
- Calculate power and energy


ewr_section_categories.csv
ngonye_flow_daily_ewrx.csv
headloss_leftchannel.csv
headloss_canal.csv


## Parameters

In [1]:
input_data='./input_data/'
output_data='./output_data/'

In [2]:
category_set_name='Recommendation 1'

## Libraries

In [3]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## EWR Assurance Category Sets
Load the available EWR assurance category sets

In [4]:
category_sets=pd.read_csv(input_data + "ewr_section_categories.csv")
category_sets

,Section,Recommendation 1,All Cs,All Bs
0,A,B,C,B
1,C,B,C,B
2,D,B,C,B
3,E,B,C,B
4,FG,C,C,B


## EWR Assurance Category Set
Select the category set given as a variable in  the setup above

In [5]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
category_set

,Section,Category
0,A,B
1,C,B
2,D,B
3,E,B
4,FG,C


## Functions

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [6]:
daily=pd.read_csv(output_data + "ngonye_flow_daily_ewrx.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,Volume,EWRRefExceedance
Date,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,1,1,0.009602,0.50
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,1,2,0.009602,0.50
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,1,3,0.009602,0.50
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,1,4,0.009602,0.50
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,1,5,0.009602,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,0.017541,0.82
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,0.017541,0.82
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,0.017541,0.82


In [7]:
ewr_flow_sets=pd.read_csv(input_data + "ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
ewr_flow_sets

Month  Wet  Mod_Wet  Normal  Mod_Dry  Dry
Section Category                                           
A       A            10   75       70      60       47   38
        A            11   73       69      58       45   37
        A            12   77       72      62       48   40
        A             1   83       77      66       52   46
        A             2   90       81      69       56   51
...                 ...  ...      ...     ...      ...  ...
FG      D             5   76       76      66       49   42
        D             6   70       57      46       39   31
        D             7   64       53      41       31   27
        D             8   51       43      32       25   22
        D             9   36       31      23       17   15

[240 rows x 6 columns]

## Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [8]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section','Month'])
ewrs_A=ewrs.loc['A']
ewrs_C=ewrs.loc['C']
ewrs_D=ewrs.loc['D']
ewrs_E=ewrs.loc['E']
ewrs_FG=ewrs.loc['FG']

ewrs_A


,Category,Wet,Mod_Wet,Normal,Mod_Dry,Dry
Month,,,,,,
10,B,44,41,37,31,24
11,B,42,40,36,29,23
12,B,44,42,38,31,26
1,B,49,46,41,34,29
2,B,55,49,43,36,31
3,B,68,59,49,41,36
4,B,84,70,54,45,40
5,B,71,65,55,47,42
6,B,67,62,53,45,39


## Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [9]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

In [10]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 

## EWR Flows
Calculate the required flows by channel

In [11]:

daily['EWRChannelA']=ewrs_A.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelC']=ewrs_C.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelD']=ewrs_D.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelE']=ewrs_E.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelFG']=ewrs_FG.lookup(daily['Month'],daily['EWRBandLabel'])


In [12]:
daily['EWRTotal']=daily['EWRChannelA']+daily['EWRChannelC']+daily['EWRChannelD']+daily['EWRChannelE']+daily['EWRChannelFG']
daily['EWRProportion']=daily['EWRTotal']/daily['Flow']

In [13]:
daily['FlowCanal']=np.where(daily['Flow']-daily['EWRTotal']>1100,1100,daily['Flow']-daily['EWRTotal'])
daily['FlowSpill']=np.where(daily['Flow']-daily['EWRTotal']<=1100,0,daily['Flow']-daily['EWRTotal']-1100)

daily['SpillChannelA']=daily['FlowSpill']*(daily['EWRChannelA']/daily['EWRTotal'])
daily['SpillChannelC']=daily['FlowSpill']*(daily['EWRChannelC']/daily['EWRTotal'])
daily['SpillChannelD']=daily['FlowSpill']*(daily['EWRChannelD']/daily['EWRTotal'])
daily['SpillChannelE']=daily['FlowSpill']*(daily['EWRChannelE']/daily['EWRTotal'])
daily['SpillChannelFG']=daily['FlowSpill']*(daily['EWRChannelFG']/daily['EWRTotal'])

daily['FlowChannelA']=daily['SpillChannelA']+daily['EWRChannelA']
daily['FlowChannelC']=daily['SpillChannelC']+daily['EWRChannelC']
daily['FlowChannelD']=daily['SpillChannelD']+daily['EWRChannelD']
daily['FlowChannelE']=daily['SpillChannelE']+daily['EWRChannelE']

daily['TmpA']=np.where(daily['FlowChannelA']>100,daily['FlowChannelA']-100,0)
daily['SpillChannelA']=daily['SpillChannelA']-daily['TmpA']
daily['FlowChannelA']=daily['FlowChannelA']-daily['TmpA']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpA']

daily['TmpC']=np.where(daily['FlowChannelC']>18,daily['FlowChannelC']-18,0)
daily['SpillChannelC']=daily['SpillChannelC']-daily['TmpC']
daily['FlowChannelC']=daily['FlowChannelC']-daily['TmpC']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpC']

daily['TmpD']=np.where(daily['FlowChannelD']>25,daily['FlowChannelD']-25,0)
daily['SpillChannelD']=daily['SpillChannelD']-daily['TmpD']
daily['FlowChannelD']=daily['FlowChannelD']-daily['TmpD']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpD']

daily['TmpE']=np.where(daily['FlowChannelE']>80,daily['FlowChannelE']-80,0)
daily['SpillChannelE']=daily['SpillChannelE']-daily['TmpE']
daily['FlowChannelE']=daily['FlowChannelE']-daily['TmpE']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpE']

daily['FlowChannelFG']=daily['SpillChannelFG']+daily['EWRChannelFG']
daily=daily.drop(['TmpA','TmpC','TmpD','TmpE'],axis=1)

daily['FlowLeftChannel']=daily['FlowChannelA']+daily['FlowChannelC']+daily['FlowChannelD']+daily['FlowCanal']


\begin{equation*}
Tailwater Level=6.0977*log(Q)^2-27.598*log(Q)+995.82
\end{equation*}

In [14]:
def headpondLevel(flow):
    if flow<5000:
        return 990.0
    elif flow<7500:
        return 0.000081307*flow+989.6
    else:
        return 0.000240*flow+988.4

In [15]:
def tailwaterLevel(flow):
    log_flow=math.log10(flow)
    return (6.0977*(log_flow**2))-(27.598*log_flow)+995.82

In [16]:
daily['LevelTailwater']=daily['Flow'].apply(tailwaterLevel)
daily['LevelHeadpond']=daily['Flow'].apply(headpondLevel)

In [17]:
headloss_leftchannel=pd.read_csv(input_data + "headloss_leftchannel.csv")
fHeadlossLeftChannel = interpolate.interp1d(headloss_leftchannel['FlowLeftChannel'], headloss_leftchannel['HeadlossLeftChannel'])
daily['HeadlossLeftChannel']=daily['FlowLeftChannel'].apply(fHeadlossLeftChannel)


In [18]:
headloss_canal=pd.read_csv(input_data + "headloss_canal.csv")
fHeadlossCanal = interpolate.interp1d(headloss_canal['FlowCanal'], headloss_canal['HeadlossCanal'])

daily['HeadlossCanal']=daily['FlowCanal'].apply(fHeadlossCanal)
daily['LevelForebay']=daily['LevelHeadpond']-daily['HeadlossLeftChannel']-daily['HeadlossCanal']

In [19]:
daily['Turbines']=daily.apply(lambda x: math.ceil(x['FlowCanal'] / 275),axis=1)
daily.loc[daily['FlowCanal']<=50,'Turbines']=0

In [20]:
daily['FlowTurbine1']=np.where(daily['Turbines']>=1,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)

\begin{equation*}
Headloss Powerhouse=0.00000702Q^2 + 0.00002226Q - 0.00207422
\end{equation*}

In [21]:
def turbineHeadloss(flow):
    if np.isnan(flow):
        return np.nan
    elif flow==0:
        return 0
    else:
        return 0.00000702*flow**2+0.00002226*flow-0.00207422

In [22]:
daily['HeadlossTurbine1']=daily['FlowTurbine1'].apply(turbineHeadloss)
daily['HeadlossTurbine2']=daily['FlowTurbine2'].apply(turbineHeadloss)
daily['HeadlossTurbine3']=daily['FlowTurbine3'].apply(turbineHeadloss)
daily['HeadlossTurbine4']=daily['FlowTurbine4'].apply(turbineHeadloss)

In [23]:
daily['HeadTurbine1']=np.where(daily['FlowTurbine1']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine1'],np.nan)
daily['HeadTurbine2']=np.where(daily['FlowTurbine2']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine2'],np.nan)
daily['HeadTurbine3']=np.where(daily['FlowTurbine3']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine3'],np.nan)
daily['HeadTurbine4']=np.where(daily['FlowTurbine4']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine4'],np.nan)


In [24]:
def turbineEfficiency(q,h):
    if np.isnan(h) or np.isnan(q):
        return np.nan
    
    if h<=10:
        return 0
    
    if q==0:
        return 0
    elif q<53.5:
        return 0.77
    elif q<55:
        return 0.78
    elif q<58.5:
        return 0.79
    elif q<62:
        return 0.80
    elif q<64:
        return 0.81
    elif q<66.5:
        return 0.82
    elif q<70:
        return 0.83
    elif q<72:
        return 0.84
    elif q<74.5:
        return 0.85
    elif q<82.5:
        return 0.86
    elif q<91.5:
        return 0.87
    elif q<100.5:
        return 0.88
    elif q<109.72:
        return 0.89
    elif q<121.77:
        return 0.90
    elif q<146.8:
        return 0.91
    elif q<169.545:
        return 0.92
    elif q<173.1:
        if h<24.65:
            return 0.92
        else:
            return 0.93
    elif q<199.8:
        return 0.93
    elif q<211.2:
        if h<23:
            return 0.93
        else:
            return 0.94
    elif q<247.05:
        return 0.94
    elif q<250:
        if h<24.16:
            return 0.94
        else:
            return 0.95
    elif q<260:
        if h<23.6:
            return 0.94
        else:
            return 0.95
    elif q<275:
        if h<22.5:
            return 0.94
        else:
            return 0.95
    else:
        return -0.0015*h**2 + 0.056*h + 0.3995

In [25]:
daily['EffTurbine1']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine1'],x['HeadTurbine1']),axis=1)
daily['EffTurbine2']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine2'],x['HeadTurbine2']),axis=1)
daily['EffTurbine3']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine3'],x['HeadTurbine3']),axis=1)
daily['EffTurbine4']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine4'],x['HeadTurbine4']),axis=1)

In [26]:
daily['LoadFactorTurbine1']=daily['FlowTurbine1']/250
daily['LoadFactorTurbine2']=daily['FlowTurbine2']/250
daily['LoadFactorTurbine3']=daily['FlowTurbine3']/250
daily['LoadFactorTurbine4']=daily['FlowTurbine4']/250

In [27]:
def generatorEff(load_fact):
    if np.isnan(load_fact):
        return np.nan
    
    if load_fact==0:
        return 0
    elif load_fact<=0.45:
        return 0.96
    elif load_fact<=0.85:
        return 0.97
    else:
        return 0.98

In [28]:
daily['EffGen1']=daily['LoadFactorTurbine1'].apply(generatorEff)
daily['EffGen2']=daily['LoadFactorTurbine2'].apply(generatorEff)
daily['EffGen3']=daily['LoadFactorTurbine3'].apply(generatorEff)
daily['EffGen4']=daily['LoadFactorTurbine4'].apply(generatorEff)

In [77]:
daily['PowerTurbine1']=daily['FlowTurbine1']*daily['HeadTurbine1']*daily['EffTurbine1']*daily['EffGen1']*9.81/1000
daily['PowerTurbine2']=daily['FlowTurbine2']*daily['HeadTurbine2']*daily['EffTurbine2']*daily['EffGen2']*9.81/1000
daily['PowerTurbine3']=daily['FlowTurbine3']*daily['HeadTurbine3']*daily['EffTurbine3']*daily['EffGen3']*9.81/1000
daily['PowerTurbine4']=daily['FlowTurbine4']*daily['HeadTurbine4']*daily['EffTurbine4']*daily['EffGen4']*9.81/1000
daily['Power']=np.nan_to_num(daily['PowerTurbine1'])+np.nan_to_num(daily['PowerTurbine2'])+np.nan_to_num(daily['PowerTurbine3'])+np.nan_to_num(daily['PowerTurbine4'])
daily['Energy']=daily['Power']*24
daily['PerfMwPerCumec']=daily['Power']/daily['Flow']

In [78]:
display(daily.loc['19570707'])

#display(daily.info())

LaggedDate        1957-07-18
VicFalls                 744
Conversion          0.885525
Flow                 658.831
Exceedance             0.456
                     ...    
PowerTurbine3            NaN
PowerTurbine4            NaN
Power                 97.562
Energy               2341.49
PerfMwPerCumec      0.148084
Name: 1957-07-07 00:00:00, Length: 73, dtype: object

In [80]:

monthly=daily.groupby("MonthId").mean().drop(['VicFalls','Conversion','Day','WaterDay','Volume','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(180*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2016.10,186.117272,0.973290,2016.0,10.0,2016.0,1.0,31.000000,4.000000,3.000000,2.000000,...,NaN,24.234289,0.129721,29.727187,19.160688,1,1,18030.310753,31,0.134635
2016.11,186.562065,0.968767,2016.0,11.0,2016.0,2.0,25.000000,4.000000,2.000000,2.000000,...,NaN,28.368567,0.149929,39.951666,18.252168,1,1,20425.368221,30,0.157603
2016.12,369.487694,0.701516,2016.0,12.0,2016.0,3.0,29.387097,4.677419,3.000000,2.000000,...,NaN,64.584072,0.174341,106.440111,39.676509,2,1,48050.549516,31,0.358800
2017.01,851.896141,0.394581,2017.0,1.0,2016.0,4.0,44.387097,6.677419,5.354839,3.677419,...,42.706262,138.547201,0.161604,176.581540,67.050590,4,2,103079.117408,31,0.769707
2017.02,1236.957587,0.273679,2017.0,2.0,2016.0,5.0,49.000000,8.000000,7.000000,5.000000,...,44.975472,179.901886,0.147109,191.580502,170.377502,4,4,120894.067624,28,0.999455
2017.03,2420.771572,0.145774,2017.0,3.0,2016.0,6.0,49.000000,9.000000,9.000000,9.000000,...,38.453020,153.812080,0.074276,178.811253,117.791222,4,4,114436.187190,31,0.854512
2017.04,3419.680887,0.063633,2017.0,4.0,2016.0,7.0,70.000000,14.000000,18.000000,25.000000,...,31.885276,127.541104,0.037906,141.584371,117.027415,4,4,91829.594526,30,0.708562
2017.05,2194.251076,0.156387,2017.0,5.0,2016.0,8.0,55.000000,13.000000,13.000000,17.000000,...,39.663234,158.652936,0.074586,175.077861,143.051018,4,4,118037.784680,31,0.881405
2017.06,1117.935234,0.312067,2017.0,6.0,2016.0,9.0,53.000000,12.000000,12.000000,13.000000,...,44.199018,154.903353,0.140692,186.152604,95.374353,4,2,111530.414134,30,0.860574


In [81]:
yearly=daily.groupby("WaterYear").agg(
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(180*24*365)


,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,
1924,189.157311,0.000000,822430.014482,0.521582
1925,193.007705,22.234928,764130.985329,0.484609
1926,190.150540,20.601468,852210.726707,0.540468
1927,188.620240,22.380456,821658.147487,0.521092
1928,192.069591,22.999942,743850.914635,0.471747
...,...,...,...,...
2012,188.637330,24.999747,839359.810899,0.532319
2013,190.913798,20.474417,921538.757982,0.584436
2014,191.718633,24.737908,879295.099141,0.557645


In [82]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


In [91]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy']/1000,(1-edc.index)*100)
edc

,Energy
0.00,1011.671696
0.05,951.920099
0.10,916.186352
0.15,897.243391
0.20,886.000826
0.25,874.222374
0.30,864.863012
0.35,852.788342
0.40,847.096845
0.45,839.159448


In [85]:
daily.to_csv(output_data+'p_and_e_daily.csv')

In [86]:
monthly.to_csv(output_data+'p_and_e_monthly.csv')

In [87]:
yearly.to_csv(output_data+'p_and_e_yearly.csv')

In [92]:
edc.to_csv(output_data+'p_and_e_percentiles.csv')